In [326]:
import pandas as pd
import os
import fnmatch
import numpy as np
from library import start
from library import clean_tea
import datetime as dt
pd.options.display.max_columns = None
year = 'yr1718'
file = "CERTIFICATION_01.csv"
if year == 'yr1213' or year == 'yr1314':
    file = "CERTIFICATION_01.TXT"
folder = 'certification_' + year
file = teacher_datapath + '/' + file
teacher_datapath = os.path.join(start.data_path, 'tea', 'teachers', folder)
year

'yr1718'

# Import, Rename Vars, Fix Grades, Expiration as Date Time

In [327]:
certification = pd.read_csv(file, sep=",", encoding="ISO-8859-1", dtype=object)
if year > 'yr1415':
    vars_to_keep = {'PERSONID_SCRAM': 'teacher_id', 'DISTRICT': 'district', 'ROLE_CREDENTIALED_FOR': 'role',
                    'CREDENTIAL_TYPE': 'cert_type', 'CERTIFICATE_PREPARATION_ROUTE': 'cert_route',
                    'CERTIFICATE_EXPIRATION_DATE': 'expiration',
                    'CERTIFICATION_LEVEL': 'cert_level', 'CREDENTIALED_GRADES': 'cert_grades',
                    'SUBJECT_AREA': 'cert_area', 'SUBJECT': 'cert_subject'}
else:
    vars_to_keep = {'personid_SCRAM': 'teacher_id', 'DISTRICT': 'district', 'ROLE_CREDENTIALED FOR': 'role',
                    'CERTIFICATE EXPIRATION DATE': 'expiration',
                    'CREDENTIAL TYPE': 'cert_type', 'CERTIFICATE PREPARATION ROUTE': 'cert_route',
                    'CERTIFICATION LEVEL': 'cert_level', 'CREDENTIALED GRADES': 'cert_grades',
                    'SUBJECT AREA': 'cert_area', 'SUBJECT': 'cert_subject'}
certification = clean_tea.filter_and_rename_cols(certification, vars_to_keep)
certification['cert_grades'] = certification['cert_grades'].replace({'Grades ':''}, regex = True)
grades = {'12-Aug': '8-12', '12-Jul': '7-12',
          '12-Jun': '6-12', '6-Jan': '1-6',
          '8-Apr': '4-8', '8-Jan': '1-8', 'EC-12': '0-12',
         'EC-4': '0-4', 'EC-6': '0-6', 'PK-12': '0-12',
         'PK-3': '0-3', 'PK-6': '0-6', 'PK-KG': '0-1'}
certification['cert_grades'] = certification['cert_grades'].replace(grades)
certification['cert_grade_low'],certification['cert_grade_high'] = certification['cert_grades'].str.split('-').str
certification['expiration'] = certification['expiration'].str[0:9]
certification['expiration']= pd.to_datetime(certification['expiration']) 
certification.sample(5)

,teacher_id,district,role,cert_type,cert_route,expiration,cert_level,cert_grades,cert_area,cert_subject,cert_grade_low,cert_grade_high
51327,V30Q1P*41,108808,Teacher,Provisional,Standard Program,NaT,Elementary,1-8,Mathematics,Mathematics,1,8
39548,233V41V36,031912,Educational Aide,Standard Paraprofessional,Paraprofessional Program,2012-02-29,Not Applicable,NaN,Other,Not Applicable,NaN,NaN
88900,V3727Y040,108906,Educational Aide,Standard Paraprofessional,Paraprofessional Program,2016-12-31,Not Applicable,NaN,Other,Not Applicable,NaN,NaN
7611,1375QLQ49,031901,Principal,Standard Professional,Standard Program,2017-01-31,Professional Service,0-12,Professional,Principal,0,12
13522,P364QV142,031901,Teacher,Probationary,Alternative Program,2006-02-03,Elementary,0-4,Bilingual Education,Bilingual Spanish,0,4


## Only keep if certification is for teaching and unexpired

In [328]:
certification = certification[certification.role == 'Teacher']
#certification['expired'] = np.where(certification.expiration < dt.date(year = 2017, month = 7, day = 1), True, False)
certification['expired'] = np.where(certification.expiration < pd.Timestamp('2017-07-01'), True, False)
certification = certification[certification.expired == False]
certification.sample(5)

,teacher_id,district,role,cert_type,cert_route,expiration,cert_level,cert_grades,cert_area,cert_subject,cert_grade_low,cert_grade_high,expired
102283,03503D540,108909,Teacher,Standard,Alternative Program,2019-03-31,Supplemental,NaN,Bilingual Education,English as a Second Language,NaN,NaN,False
164721,V3500L145,240901,Teacher,Standard,Standard Program,2017-07-31,Elementary,1-8,General Elementary (Self-Contained),Self-Contained,1,8,False
119546,V35V2V148,108911,Teacher,Provisional,Standard Program,NaT,Elementary,1-8,English Language Arts,Reading,1,8,False
68842,D31QVQ241,108904,Teacher,Standard,Alternative Program,2023-08-31,Special Education,0-12,Special Education,Generic Special Education,0,12,False
178026,D35*7*241,240903,Teacher,Standard,Certification by Exam,2018-06-30,Endorsement,NaN,Bilingual Education,Bilingual Spanish,NaN,NaN,False


## Create certified variable

In [329]:
cert_types = {'Emergency Non-Certified': False, 'Emergency Certified': True,
                  'Emergency': False, 'Emergency Teaching': False,
                  'Temporary Exemption': True, 'Temporary Teaching Certificate': False,
                  'Unknown Permit': False, 'Unknown': False,
                  'Special Assignment': True,
                  'Paraprofessional': False, 'Standard Paraprofessional': False, 'Non-renewable': False,
                  'Standard': True, 'Provisional': True,
                  'Probationary': True, 'Probationary Extension': True, 'Probationary Second Extension': True,
                   'One Year': True,
                  'Visiting International Teacher': True,
                  'Professional': True, 'Standard Professional': True}
certification['certified'] = certification['cert_type'].map(cert_types)
certification['vocational'] = np.where((certification['cert_type'] == "Vocational"), True, False)
certification.sample(10)

,teacher_id,district,role,cert_type,cert_route,expiration,cert_level,cert_grades,cert_area,cert_subject,cert_grade_low,cert_grade_high,expired,certified,vocational
152447,1373QQ337,214903,Teacher,Standard,Certification by Exam,2019-07-31,Secondary,8-12,Computer Science,Computer Science,8,12,False,True,False
53368,03QQ4F544,108902,Teacher,Standard,Certification by Exam,2017-12-31,Secondary,8-12,Science,Science - Composite,8,12,False,True,False
159715,232Q3F243,240901,Teacher,Standard,Post-Baccalaureate,2020-08-31,Elementary,0-4,General Elementary (Self-Contained),Generalist,0,4,False,True,False
167605,*35Q4P045,240903,Teacher,Provisional,Alternative Program,NaT,Special Education,0-12,Special Education,Generic Special Education,0,12,False,True,False
168945,03326L244,240903,Teacher,Standard,Alternative Program,2019-06-30,Secondary,8-12,Science,Life Science,8,12,False,True,False
116237,03455P349,108911,Teacher,Standard,Certification by Exam,2019-01-31,All Level,0-12,Special Education,Generic Special Education,0,12,False,True,False
76372,Y3400D*48,108904,Teacher,Standard,Certification by Exam,2020-12-31,Supplemental,NaN,Bilingual Education,Bilingual Ed Supplemental-Spanish,NaN,NaN,False,True,False
20068,V3664*245,031901,Teacher,Provisional,Certification by Exam,NaT,Secondary,6-12,Social Studies,Sociology,6,12,False,True,False
37046,03422V047,031911,Teacher,Standard,Standard Program,2021-04-30,Elementary,0-4,General Elementary (Self-Contained),Generalist,0,4,False,True,False
31297,V370VVP44,031905,Teacher,Standard,Standard Program,2018-02-28,Elementary,4-8,Mathematics,Mathematics,4,8,False,True,False


# Reshape long to wide

In [330]:
df = certification[['teacher_id', 'district', 
                                       'cert_area', 'cert_subject',
                                       'cert_grade_low', 'cert_grade_high']]
df['idx'] = df.groupby('teacher_id').cumcount()
certs = certification[['teacher_id','certified', 'vocational']].groupby('teacher_id').max()
df = df.merge(certs, how = 'left', on = 'teacher_id')
df['cert_area_idx'] = 'cert_area_' + df.idx.astype(str)
df['cert_subject_idx'] = 'cert_subject_' + df.idx.astype(str)
df['cert_grade_low_idx'] = 'cert_grade_low_' + df.idx.astype(str)
df['cert_grade_high_idx'] = 'cert_grade_high_' + df.idx.astype(str)
df

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,teacher_id,district,cert_area,cert_subject,cert_grade_low,cert_grade_high,idx,certified,vocational,cert_area_idx,cert_subject_idx,cert_grade_low_idx,cert_grade_high_idx
0,*30*5P341,031901,Special Education,Generic Special Education,0,12,0,True,False,cert_area_0,cert_subject_0,cert_grade_low_0,cert_grade_high_0
1,*3034L040,031901,Computer Science,Computer Information Systems,6,12,0,True,False,cert_area_0,cert_subject_0,cert_grade_low_0,cert_grade_high_0
2,*3034L040,031901,Vocational Education,Technology Education,6,12,1,True,False,cert_area_1,cert_subject_1,cert_grade_low_1,cert_grade_high_1
3,*3034L040,031901,Vocational Education,Business Education,6,12,2,True,False,cert_area_2,cert_subject_2,cert_grade_low_2,cert_grade_high_2
4,*3034L040,031901,Foreign Language,Spanish,0,12,3,True,False,cert_area_3,cert_subject_3,cert_grade_low_3,cert_grade_high_3
5,*3040V545,031901,Bilingual Education,Bilingual Spanish,1,8,0,True,False,cert_area_0,cert_subject_0,cert_grade_low_0,cert_grade_high_0
6,*3040V545,031901,General Elementary (Self-Contained),Self-Contained,1,8,1,True,False,cert_area_1,cert_subject_1,cert_grade_low_1,cert_grade_high_1
7,*3040V545,031901,General Elementary (Self-Contained),Early Childhood Education,0,1,2,True,False,cert_area_2,cert_subject_2,cert_grade_low_2,cert_grade_high_2
8,*31271Q42,031901,Vocational Education,Technology Applications,0,12,0,True,False,cert_area_0,cert_subject_0,cert_grade_low_0,cert_grade_high_0
9,*3145*048,031901,Special Education,Generic Special Education,0,12,0,True,False,cert_area_0,cert_subject_0,cert_grade_low_0,cert_grade_high_0


In [331]:
areas = df.pivot(index='teacher_id',columns='cert_area_idx', values='cert_area')
subjects = df.pivot(index='teacher_id',columns='cert_subject_idx', values='cert_subject')
low_grades = df.pivot(index='teacher_id',columns='cert_grade_low_idx', values='cert_grade_low')
high_grades = df.pivot(index='teacher_id',columns='cert_grade_high_idx', values='cert_grade_high')
teacher_cert = pd.concat([areas, subjects, low_grades, high_grades], axis = 1)
teacher_cert = teacher_cert[['cert_area_0', 'cert_subject_0', 'cert_grade_low_0', 'cert_grade_high_0',
                            'cert_area_1', 'cert_subject_1', 'cert_grade_low_1', 'cert_grade_high_1',
                            'cert_area_2', 'cert_subject_2', 'cert_grade_low_2', 'cert_grade_high_2',
                            'cert_area_3', 'cert_subject_3', 'cert_grade_low_3', 'cert_grade_high_3',
                            'cert_area_4', 'cert_subject_4', 'cert_grade_low_4', 'cert_grade_high_4',
                            'cert_area_5', 'cert_subject_5', 'cert_grade_low_5', 'cert_grade_high_5',
                            'cert_area_6', 'cert_subject_6', 'cert_grade_low_6', 'cert_grade_high_6',
                            'cert_area_7', 'cert_subject_7', 'cert_grade_low_7', 'cert_grade_high_7',
                            'cert_area_8', 'cert_subject_8', 'cert_grade_low_8', 'cert_grade_high_8',
                            'cert_area_9', 'cert_subject_9', 'cert_grade_low_9', 'cert_grade_high_9',
                            'cert_area_10', 'cert_subject_10', 'cert_grade_low_10', 'cert_grade_high_10',
                            'cert_area_11', 'cert_subject_11', 'cert_grade_low_11', 'cert_grade_high_11',
                            'cert_area_12', 'cert_subject_12', 'cert_grade_low_12', 'cert_grade_high_12',
                            'cert_area_13', 'cert_subject_13', 'cert_grade_low_13', 'cert_grade_high_13',
                            'cert_area_14', 'cert_subject_14', 'cert_grade_low_14', 'cert_grade_high_14']]
teacher_cert = teacher_cert.merge(df[['teacher_id', 'district', 'certified', 'vocational']], 
                                  left_index = True, right_on = 'teacher_id')
teacher_cert

,cert_area_0,cert_subject_0,cert_grade_low_0,cert_grade_high_0,cert_area_1,cert_subject_1,cert_grade_low_1,cert_grade_high_1,cert_area_2,cert_subject_2,cert_grade_low_2,cert_grade_high_2,cert_area_3,cert_subject_3,cert_grade_low_3,cert_grade_high_3,cert_area_4,cert_subject_4,cert_grade_low_4,cert_grade_high_4,cert_area_5,cert_subject_5,cert_grade_low_5,cert_grade_high_5,cert_area_6,cert_subject_6,cert_grade_low_6,cert_grade_high_6,cert_area_7,cert_subject_7,cert_grade_low_7,cert_grade_high_7,cert_area_8,cert_subject_8,cert_grade_low_8,cert_grade_high_8,cert_area_9,cert_subject_9,cert_grade_low_9,cert_grade_high_9,cert_area_10,cert_subject_10,cert_grade_low_10,cert_grade_high_10,cert_area_11,cert_subject_11,cert_grade_low_11,cert_grade_high_11,cert_area_12,cert_subject_12,cert_grade_low_12,cert_grade_high_12,cert_area_13,cert_subject_13,cert_grade_low_13,cert_grade_high_13,cert_area_14,cert_subject_14,cert_grade_low_14,cert_grade_high_14,teacher_id,district,certified,vocational
19500,English Language Arts,Reading,1,8,General Elementary (Self-Contained),Self-Contained,1,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,*30*1FV49,108902,True,False
19501,English Language Arts,Reading,1,8,General Elementary (Self-Contained),Self-Contained,1,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,*30*1FV49,108902,True,False
53863,Bilingual Education,Bilingual Ed Supplemental-Spanish,NaN,NaN,General Elementary (Self-Contained),Generalist,0,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,*30*2**40,214901,True,False
53864,Bilingual Education,Bilingual Ed Supplemental-Spanish,NaN,NaN,General Elementary (Self-Contained),Generalist,0,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,*30*2**40,214901,True,False
0,Special Education,Generic Special Education,0,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,*30*5P341,031901,True,False
22848,Bilingual Education,Bilingual Spanish,1,8,General Elementary (Self-Contained),Self-Contained,1,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,*30*QF546,108904,True,False
22849,Bilingual Education,Bilingual Spanish,1,8,General Elementary (Self-Contained),Self-Contained,1,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,*30*QF546,108904,True,False
22850,Bilingual Education,Bilingual Spanish,1,8,General Elementary (Self-Contained),Self-Contained,1,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,*3001L344,108904,True,False
22851,Bilingual Education,Bilingual Spanish,1,8,General Elementary (Self-Contained),Self-Contained,1,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,*3001

In [332]:
file = teacher_datapath + '/' + 'TEACHER_CLASS_01.TXT'
courses = pd.read_csv(file, sep=",", encoding="ISO-8859-1", dtype=object)
courses.columns
courses

,REGION NUMBER,COUNTY NUMBER,COUNTY NAME,DISTRICT NUMBER,DISTRICT NAME,DISTRICT CHARTER TYPE CODE,DISTRICT CHARTER TYPE NAME,CAMPUS NUMBER,CAMPUS NAME,CAMPUS CHARTER TYPE CODE,CAMPUS CHARTER TYPE NAME,CAMPUS ADDRESS,CAMPUS CITY,CAMPUS STATE,CAMPUS ZIP,CAMPUS EMAIL,SCRAMBLED UNIQUE ID,FIRST NAME,MIDDLE NAME,LAST NAME,ROLE CODE,ROLE NAME,CLASS NUMBER,CLASS NAME,CLASS TYPE CODE,CLASS TYPE NAME,SUBJECT AREA CODE,SUBJECT AREA NAME,SUBJECT CODE,SUBJECT NAME,GRADE LEVEL CODE,GRADE LEVEL NAME,ADVANCED COURSE,PARTIAL FULL TIME EQUIVALENT
0,01,031,CAMERON COUNTY,031901,BROWNSVILLE ISD,00,NOT A CHARTER DISTRICT,031901001,HANNA EARLY COLLEGE H S,00,NOT A CHARTER SCHOOL,2615 PRICE RD,BROWNSVILLE,TX,78521-2430,lambarri@bisd.us,*32QVD441,JOSEFINA,NaN,TORRES-MORENO,087,TEACHER,03440200,LANG O/T ENGLISH II - SPANISH,01,ACADEMIC ACHIEVEMENT COURSE,07,FOREIGN LANGUAGE,44,SPANISH,45,GRADES 9-12,N,0.2858
1,01,031,CAMERON COUNTY,031901,BROWNSVILLE ISD,00,NOT A CHARTER DISTRICT,031901001,HANNA EARLY COLLEGE H S,00,NOT A CHARTER SCHOOL,2615 PRICE RD,BROWNSVILLE,TX,78521-2430,lambarri@bisd.us,*32QVD441,JOSEFINA,NaN,TORRES-MORENO,087,TEACHER,03440300,LANG O/T ENGLISH III - SPANISH,01,ACADEMIC ACHIEVEMENT COURSE,07,FOREIGN LANGUAGE,44,SPANISH,45,GRADES 9-12,N,0.1429
2,01,031,CAMERON COUNTY,031901,BROWNSVILLE ISD,00,NOT A CHARTER DISTRICT,031901001,HANNA EARLY COLLEGE H S,00,NOT A CHARTER SCHOOL,2615 PRICE RD,BROWNSVILLE,TX,78521-2430,lambarri@bisd.us,*32QVD441,JOSEFINA,NaN,TORRES-MORENO,087,TEACHER,03440400,LANG O/T ENGLISH IV - SPANISH,01,ACADEMIC ACHIEVEMENT COURSE,07,FOREIGN LANGUAGE,44,SPANISH,45,GRADES 9-12,N,0.5716
3,01,031,CAMERON COUNTY,031901,BROWNSVILLE ISD,00,NOT A CHARTER DISTRICT,031901001,HANNA EARLY COLLEGE H S,00,NOT A CHARTER SCHOOL,2615 PRICE RD,BROWNSVILLE,TX,78521-2430,lambarri@bisd.us,*332QP124,JAMESON,SHERMAN,WICKS,087,TEACHER,PES00000,PE SUB ATHLETICS 1,01,ACADEMIC ACHIEVEMENT COURSE,06,PHYSICAL ED. & HEALTH,82,PHYSICAL EDUCATION,45,GRADES 9-12,N,0.2407
4,01,031,CAMERON COUNTY,031901,BROWNSVILLE ISD,00,NOT A CHARTER DISTRICT,031901001,HANNA EARLY COLLEGE H S,00,NOT A CHARTER SCHOOL,2615 PRICE RD,BROWNSVILLE,TX,78521-2430,lambarri@bisd.us,*332QP124,JAMESON,SHERMAN,WICKS,087,TEACHER,PES00001,PE SUB ATHLETICS 2,01,ACADEMIC ACHIEVEMENT COURSE,06,PHYSICAL ED. & HEALTH,82,PHYSICAL EDUCATION,45,GRADES 9-12,N,0.2407
5,01,031,CAMERON COUNTY,031901,BROWNSVILLE ISD,00,NOT A CHARTER DISTRICT,031901001,HANNA EARLY COLLEGE H S,00,NOT A CHARTER SCHOOL,2615 PRICE RD,BROWNSVILLE,TX,78521-2430,lambarri@bisd.us,*332QP124,JAMESON,SHERMAN,WICKS,087,TEACHER,PES00002,PE SUB ATHLETICS 3,01,ACADEMIC ACHIEVEMENT COURSE,06,PHYSICAL ED. & HEALTH,82,PHYSICAL EDUCATION,45,GRADES 9-12,N,0.2407
6,01,031,CAMERON COUNTY,031901,BROWNSVILLE ISD,00,NOT A CHARTER DISTRICT,031901001,HANNA EARLY COLLEGE H S,00,NOT A CHARTER SCHOOL,2615 PRICE RD,BROWNSVILLE,TX,78521-2430,lambarri@bisd.us,*332QP124,JAMESON,SHERMAN,WICKS,087,TEACHER,PES00003,PE SUB ATHLETICS 4,01,ACADEMIC ACHIEVEMENT COURSE,06,PHYSICAL ED. & HEALTH,82,PHYSICAL EDUCATION,45,GRADES 9-12,N,0.2407
7,01,031,CAMERON COUNTY,031901,BROWNSVILLE ISD,00,NOT A CHARTER DISTRICT,031901001,HANNA EARLY COLLEGE H S,00,NOT A CHARTER SCHOOL,2615 PRICE RD,BROWNSVILLE,TX,78521-2430,lambarri@bisd.us,*35*4P445,MARIO,ALBERTO,MARTINEZ,087,TEACHER,03340400,WORLD HISTORY STUDIES (W HIST),01,ACADEMIC ACHIEVEMENT COURSE,05,SOCIAL STUDIES,34,HISTORY,45,GRADES 9-12,N,1.0001
8,01,031,CAMERON COUNTY,031901,BROWNSVILLE ISD,00,NOT A CHARTER DISTRICT,031901001,HANNA EARLY COLLEGE H S,00,NOT A CHARTER SCHOOL,2615 PRICE RD,BROWNSVILLE,TX,78521-2430,lambarri@bisd.us,*35001P47,DALBERTO,N,GARZA,087,TEACHER,13011400,BUSIN INFO MGT I,01,ACADEMIC ACHIEVEMENT COURSE,11,CAREER & TECHNOLOGY EDUCATION,79,BUSINESS MANAGEMENT & ADMIN,45,GRADES 9-12,N,0.5572
9,01,031,CAMERON COUNTY,031901,BROWNSVILLE ISD,00,NOT A CHARTER DISTRICT,031901001,HANNA EARLY COLLEGE H S,00,NOT A CHARTER SCHOOL,2615 PRICE RD,BROWNSVILLE,TX,78521-2430,lambarri@bisd.us,*35001P47,DAL

In [333]:
courses['ROLE CODE'].value_counts()

087    109392
047        32
Name: ROLE CODE, dtype: int64

In [334]:
vars_to_keep = {'SCRAMBLED UNIQUE ID': 'teacher_id',
                'DISTRICT NUMBER': 'district', 'CAMPUS NUMBER': 'campus',
               'SUBJECT AREA NAME': 'course_area', 'SUBJECT NAME': 'course_subject',
               'GRADE LEVEL NAME': 'grades'}
courses = clean_tea.filter_and_rename_cols(courses, vars_to_keep)
courses

,teacher_id,district,campus,course_area,course_subject,grades
0,*32QVD441,031901,031901001,FOREIGN LANGUAGE,SPANISH,GRADES 9-12
1,*32QVD441,031901,031901001,FOREIGN LANGUAGE,SPANISH,GRADES 9-12
2,*32QVD441,031901,031901001,FOREIGN LANGUAGE,SPANISH,GRADES 9-12
3,*332QP124,031901,031901001,PHYSICAL ED. & HEALTH,PHYSICAL EDUCATION,GRADES 9-12
4,*332QP124,031901,031901001,PHYSICAL ED. & HEALTH,PHYSICAL EDUCATION,GRADES 9-12
5,*332QP124,031901,031901001,PHYSICAL ED. & HEALTH,PHYSICAL EDUCATION,GRADES 9-12
6,*332QP124,031901,031901001,PHYSICAL ED. & HEALTH,PHYSICAL EDUCATION,GRADES 9-12
7,*35*4P445,031901,031901001,SOCIAL STUDIES,HISTORY,GRADES 9-12
8,*35001P47,031901,031901001,CAREER & TECHNOLOGY EDUCATION,BUSINESS MANAGEMENT & ADMIN,GRADES 9-12
9,*35001P47,031901,031901001,CAREER & TECHNOLOGY EDUCATION,BUSINESS MANAGEMENT & ADMIN,GRADES 9-12


In [335]:
course_cert = courses.merge(teacher_cert, left_on = 'teacher_id', right_on = 'teacher_id', how = 'left')
course_cert

,teacher_id,district_x,campus,course_area,course_subject,grades,cert_area_0,cert_subject_0,cert_grade_low_0,cert_grade_high_0,cert_area_1,cert_subject_1,cert_grade_low_1,cert_grade_high_1,cert_area_2,cert_subject_2,cert_grade_low_2,cert_grade_high_2,cert_area_3,cert_subject_3,cert_grade_low_3,cert_grade_high_3,cert_area_4,cert_subject_4,cert_grade_low_4,cert_grade_high_4,cert_area_5,cert_subject_5,cert_grade_low_5,cert_grade_high_5,cert_area_6,cert_subject_6,cert_grade_low_6,cert_grade_high_6,cert_area_7,cert_subject_7,cert_grade_low_7,cert_grade_high_7,cert_area_8,cert_subject_8,cert_grade_low_8,cert_grade_high_8,cert_area_9,cert_subject_9,cert_grade_low_9,cert_grade_high_9,cert_area_10,cert_subject_10,cert_grade_low_10,cert_grade_high_10,cert_area_11,cert_subject_11,cert_grade_low_11,cert_grade_high_11,cert_area_12,cert_subject_12,cert_grade_low_12,cert_grade_high_12,cert_area_13,cert_subject_13,cert_grade_low_13,cert_grade_high_13,cert_area_14,cert_subject_14,cert_grade_low_14,cert_grade_high_14,district_y,certified,vocational
0,*32QVD441,031901,031901001,FOREIGN LANGUAGE,SPANISH,GRADES 9-12,Bilingual Education,Bilingual Spanish,0,4,English Language Arts,English Language Arts and Reading,4,8,Bilingual Education,English as a Second Language,NaN,NaN,General Elementary (Self-Contained),Generalist,0,4,Foreign Language,Spanish,6,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,031901,True,False
1,*32QVD441,031901,031901001,FOREIGN LANGUAGE,SPANISH,GRADES 9-12,Bilingual Education,Bilingual Spanish,0,4,English Language Arts,English Language Arts and Reading,4,8,Bilingual Education,English as a Second Language,NaN,NaN,General Elementary (Self-Contained),Generalist,0,4,Foreign Language,Spanish,6,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,031901,True,False
2,*32QVD441,031901,031901001,FOREIGN LANGUAGE,SPANISH,GRADES 9-12,Bilingual Education,Bilingual Spanish,0,4,English Language Arts,English Language Arts and Reading,4,8,Bilingual Education,English as a Second Language,NaN,NaN,General Elementary (Self-Contained),Generalist,0,4,Foreign Language,Spanish,6,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,031901,True,False
3,*32QVD441,031901,031901001,FOREIGN LANGUAGE,SPANISH,GRADES 9-12,Bilingual Education,Bilingual Spanish,0,4,English Language Arts,English Language Arts and Reading,4,8,Bilingual Education,English as a Second Language,NaN,NaN,General Elementary (Self-Contained),Generalist,0,4,Foreign Language,Spanish,6,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,031901,True,False
4,*32QVD441,031901,031901001,FOREIGN LANGUAGE,SPANISH,GRADES 9-12,Bilingual Education,Bilingual Spanish,0,4,English Language Arts,English Language Arts and Reading,4,8,Bilingual Education,English as a Second Language,NaN,NaN,General Elementary (Self-Contained),Generalist,0,4,Foreign Language,Spanish,6,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,031901,True,False
5,*32QVD441,031901,031901001,FOREIGN LANGUAGE,SPANISH,GRADES 9-12,Bilingual Education,Bilingual Spanish,0,4,English Language Arts,English Language Arts and Reading,4,8,Bilingual Education,English as a Second Language,NaN,NaN,General Elementary (Self-Contained),Generalist,0,4,Foreign Language,Spanish,6,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,031901,True,False
6,*32QVD441,031901,031901001,FOREIGN LANGUAGE,SPAN